#### Data engineering step

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import plotly.express as px 
import folium
import geopandas as gpd

year_dict = {
    '1516':2016,
    '1617':2017,
    '1718':2018,
    '1819':2019,
    '1920':2020,
    '2021':2021,
    '2122':2022
}
year_dict2 = {v: k for k, v in year_dict.items()}

fips_dict = {
    'Baker County':1,
    'Benton County':3,
    'Clackamas County':5,
    'Clatsop County':7,
    'Columbia County':9,
    'Coos County':11,
    'Crook County':13,
    'Curry County':15,
    'Deschutes County':17,
    'Douglas County':19,
    'Gilliam County':21,
    'Grant County':23,
    'Harney County':25,
    'Hood River County':27,
    'Jackson County':29,
    'Jefferson County':31,
    'Josephine County':33,
    'Klamath County':35,
    'Lake County':37,
    'Lane County':39,
    'Lincoln County':41,
    'Linn County':43,
    'Malheur County':45,
    'Marion County':47,
    'Morrow County':49,
    'Multnomah County':51,
    'Polk County':53,
    'Sherman County':55,
    'Tillamook County':57,
    'Umatilla County':59,
    'Union County':61,
    'Wallowa County':63,
    'Wasco County':65,
    'Washington County':67,
    'Wheeler County':69,
    'Yamhill County':71
}

# reverse the fips_dict above
fips_dict2 = {v: k for k, v in fips_dict.items()}

In [2]:
# inflow!!!
dir1 = 'inflow'

globbed_files = glob.glob(f"Raw/county{dir1}*.csv")
data = []

for csv in globbed_files:
    df = pd.read_csv(csv)
    df['yr_range'] = os.path.basename(csv)
    df['yr_range'] = df['yr_range'].str.replace(f'county{dir1}','').str.replace('.csv','', regex=True)
    df['mig_direction1'] = dir1
    data.append(df)

bigframe = pd.concat(data, ignore_index=True) 

# only get OR
bigframe = bigframe[bigframe['y2_statefips'] == 41]
bigframe.rename(columns={"y2_statefips": "y2_state"}, inplace=True)
bigframe = bigframe[bigframe['y2_countyfips'].isin(fips_dict.values())]
bigframe['agi_thou'] = bigframe['agi'] * 1000
bigframe['year1'] = '20' + bigframe['yr_range'].str.slice(stop=2)
bigframe['year2'] = '20' + bigframe['yr_range'].str.slice(start=2)
bigframe['y2_countyname'] = bigframe['y2_countyfips'].map(fips_dict2)
bigframe['y2_state'] = 'OR'
bigframe['secondary_county'] = bigframe['y1_countyname'] + ', ' + bigframe['y1_state']

# create the column that will tag the data at the metro level or single county
bigframe['unique_tag'] = np.where(
    bigframe['y1_countyname'].str.endswith('Total Migration-US and Foreign'), 
    bigframe['y2_countyname'] + ' total migration-' + bigframe['yr_range'].astype(str), 
    bigframe['y2_countyname'] + '-' + bigframe['y1_countyname'] + '-' + bigframe['y1_state'] + '-' + bigframe['yr_range'].astype(str) 
)

bigframe = bigframe[[
    'y1_countyname',
    'y1_state',
    'y2_countyname',
    'y2_state',
    'n2',
    'agi',
    'agi_thou',
    'yr_range',
    'year1',
    'year2',
    'mig_direction1',
    'unique_tag',
    'secondary_county'
]]

# drop rows that aren't either the single county counts or the total county migration count
bigframe = bigframe[~bigframe['y1_countyname'].str.contains("Non-migrants")]
bigframe = bigframe[~bigframe['y1_countyname'].str.contains("Total Migration-Same State")]
bigframe = bigframe[~bigframe['y1_countyname'].str.contains("Total Migration-Different State")]
bigframe = bigframe[~bigframe['y1_countyname'].str.contains("Total Migration-Foreign")]
bigframe = bigframe[~bigframe['y1_countyname'].str.endswith("Total Migration-US")]
bigframe = bigframe[~bigframe['y1_countyname'].str.contains('Other flows - ')]
bigframe = bigframe[~bigframe['y1_countyname'].str.startswith('Foreign - ')]

bigframe.rename(columns={
    "n2": "n2_inflow", 
    "agi": "agi_perThou_inflow",
    "agi_thou":"agi_inflow"
}, inplace=True)


bigframe.to_csv("inflow_ALL.csv")
bigframe.head(20)
# print('complete!')

,y1_countyname,y1_state,y2_countyname,y2_state,n2_inflow,agi_perThou_inflow,agi_inflow,yr_range,year1,year2,mig_direction1,unique_tag,secondary_county
61818,Baker County Total Migration-US and Foreign,OR,Baker County,OR,765,17750,17750000,1718,2017,2018,inflow,Baker County total migration-1718,"Baker County Total Migration-US and Foreign, OR"
61824,Union County,OR,Baker County,OR,88,1898,1898000,1718,2017,2018,inflow,Baker County-Union County-OR-1718,"Union County, OR"
61825,Deschutes County,OR,Baker County,OR,46,1244,1244000,1718,2017,2018,inflow,Baker County-Deschutes County-OR-1718,"Deschutes County, OR"
61826,Ada County,ID,Baker County,OR,33,613,613000,1718,2017,2018,inflow,Baker County-Ada County-ID-1718,"Ada County, ID"
61834,Benton County Total Migration-US and Foreign,OR,Benton County,OR,5323,169928,169928000,1718,2017,2018,inflow,Benton County total migration-1718,"Benton County Total Migration-US and Foreign, OR"
61840,Linn County,OR,Benton County,OR,989,24061,24061000,1718,2017,2018,inflow,Benton County-Linn County-OR-1718,"Linn County, OR"
61841,Lane County,OR,Benton County,OR,392,9846,9846000,1718,2017,2018,inflow,Benton County-Lane County-OR-1718,"Lane County, OR"
61842,Multnomah County,OR,Benton County,OR,256,7928,7928000,1718,2017,2018,inflow,Benton County-Multnomah County-OR-1718,"Multnomah County, OR"
61843,Marion County,OR,Benton County,OR,229,6252,6252000,1718,2017,2018,inflow,Benton County-Marion County-OR-1718,"Marion County, OR"
61844,Washington County,OR,Benton County,OR,138,3664,3664000,1718,2017,2018,inflow,Benton County-Washington County-OR-1718,"Washington County, OR"


In [3]:
# outflow!!!
dir2 = 'outflow'

globbed_files = glob.glob(f"Raw/county{dir2}*.csv")
data = []

for csv in globbed_files:
    df = pd.read_csv(csv)
    df['yr_range'] = os.path.basename(csv)
    df['yr_range'] = df['yr_range'].str.replace(f'county{dir2}','').str.replace('.csv','', regex=True)
    df['mig_direction2'] = dir2
    data.append(df)
    
bigframe = pd.concat(data, ignore_index=True) 

# only get OR
bigframe = bigframe[bigframe['y1_statefips'] == 41]
bigframe = bigframe[bigframe['y1_countyfips'].isin(fips_dict.values())]
bigframe['agi_thou'] = bigframe['agi'] * 1000
bigframe['year1'] = '20' + bigframe['yr_range'].str.slice(stop=2)
bigframe['year2'] = '20' + bigframe['yr_range'].str.slice(start=2)
bigframe['y1_countyname'] = bigframe['y1_countyfips'].map(fips_dict2)
bigframe['y1_state'] = 'OR'
bigframe['secondary_county'] = bigframe['y2_countyname'] + ', ' + bigframe['y2_state']

# create the column that will tag the data at the metro level or single county
bigframe['unique_tag'] = np.where(
    bigframe['y2_countyname'].str.endswith('Total Migration-US and Foreign'), 
    bigframe['y1_countyname'] + ' total migration-' + bigframe['yr_range'].astype(str), 
    bigframe['y1_countyname'] + '-' + bigframe['y2_countyname'] + '-' + bigframe['y2_state'] + '-' + bigframe['yr_range'].astype(str) 
)

bigframe = bigframe[[
    'y1_countyname',
    'y1_state',
    'y2_countyname',
    'y2_state',
    'n2',
    'agi',
    'agi_thou',
    'yr_range',
    'year1',
    'year2',
    'mig_direction2',
    'unique_tag',
    'secondary_county'
]]

# drop rows that aren't either the single county counts or the total county migration count
bigframe = bigframe[~bigframe['y2_countyname'].str.contains("Non-migrants")]
bigframe = bigframe[~bigframe['y2_countyname'].str.contains("Total Migration-Same State")]
bigframe = bigframe[~bigframe['y2_countyname'].str.contains("Total Migration-Different State")]
bigframe = bigframe[~bigframe['y2_countyname'].str.contains("Total Migration-Foreign")]
bigframe = bigframe[~bigframe['y2_countyname'].str.endswith("Total Migration-US")]
bigframe = bigframe[~bigframe['y2_countyname'].str.contains('Other flows - ')]
bigframe = bigframe[~bigframe['y2_countyname'].str.startswith('Foreign - ')]

bigframe.rename(columns={
    "n2": "n2_outflow", 
    "agi": "agi_perThou_outflow",
    "agi_thou":"agi_outflow"
}, inplace=True)

bigframe.to_csv("outflow_ALL.csv")
bigframe.head(20)
# print('complete!')

,y1_countyname,y1_state,y2_countyname,y2_state,n2_outflow,agi_perThou_outflow,agi_outflow,yr_range,year1,year2,mig_direction2,unique_tag,secondary_county
59943,Baker County,OR,Baker County Total Migration-US and Foreign,OR,620,15777,15777000,1819,2018,2019,outflow,Baker County total migration-1819,"Baker County Total Migration-US and Foreign, OR"
59948,Baker County,OR,Union County,OR,71,1476,1476000,1819,2018,2019,outflow,Baker County-Union County-OR-1819,"Union County, OR"
59949,Baker County,OR,Ada County,ID,31,964,964000,1819,2018,2019,outflow,Baker County-Ada County-ID-1819,"Ada County, ID"
59957,Benton County,OR,Benton County Total Migration-US and Foreign,OR,5479,178209,178209000,1819,2018,2019,outflow,Benton County total migration-1819,"Benton County Total Migration-US and Foreign, OR"
59962,Benton County,OR,Linn County,OR,1220,37122,37122000,1819,2018,2019,outflow,Benton County-Linn County-OR-1819,"Linn County, OR"
59963,Benton County,OR,Multnomah County,OR,371,11908,11908000,1819,2018,2019,outflow,Benton County-Multnomah County-OR-1819,"Multnomah County, OR"
59964,Benton County,OR,Washington County,OR,314,9747,9747000,1819,2018,2019,outflow,Benton County-Washington County-OR-1819,"Washington County, OR"
59965,Benton County,OR,Lane County,OR,356,9651,9651000,1819,2018,2019,outflow,Benton County-Lane County-OR-1819,"Lane County, OR"
59966,Benton County,OR,Marion County,OR,245,8643,8643000,1819,2018,2019,outflow,Benton County-Marion County-OR-1819,"Marion County, OR"
59967,Benton County,OR,Clackamas County,OR,139,4570,4570000,1819,2018,2019,outflow,Benton County-Clackamas County-OR-1819,"Clackamas County, OR"


In [4]:
# combine inflow, outflow frames
df1 = pd.read_csv('inflow_All.csv')
df2 = pd.read_csv('outflow_All.csv')

df_total = df1.merge(df2, left_on='unique_tag', right_on='unique_tag')
df_total = df_total.loc[:, ~df_total.columns.str.startswith('Unnamed')]

df_total = df_total[['unique_tag', 'secondary_county_y', 'yr_range_x', 'year1_x', 'year2_x', 'n2_inflow', 'agi_inflow', 'n2_outflow', 'agi_outflow']]
df_total.rename(columns={
    'yr_range_x':'yr_range',
    'year1_x':'year1',
    'year2_x':'year2'}, 
                inplace=True)
df_total['n2_net'] = df_total['n2_inflow'] - df_total['n2_outflow'] 
df_total['agi_net'] = df_total['agi_inflow'] - df_total['agi_outflow']

# sum up data at metro level 
df_metro = df_total[df_total['unique_tag'].str.contains('total migration-')]
df_metro = df_metro.groupby('yr_range').agg({
    'year1':'mean',
    'year2':'mean',
    'n2_inflow':'sum',
    'agi_inflow':'sum',
    'n2_outflow':'sum',
    'agi_outflow':'sum',
    'n2_net':'sum',
    'agi_net':'sum'}).reset_index()


df_metro['agi_inflow_capita'] = df_metro['agi_inflow'] / df_metro['n2_inflow']
df_metro['agi_outflow_capita'] = df_metro['agi_outflow'] / df_metro['n2_outflow']
df_metro.to_csv('metro_ALL.csv')


# # sum up data at county level
df_county_total = df_total[df_total['unique_tag'].str.contains('total migration-')]
df_county_total['arc_county'] = df_county_total['unique_tag'].map(lambda x: x.replace(' total migration-', ""))
df_county_total['arc_county'] = df_county_total['arc_county'].str[:-4]
df_county_total['agi_inflow_capita'] = df_county_total['agi_inflow'] / df_county_total['n2_inflow']
df_county_total['agi_outflow_capita'] = df_county_total['agi_outflow'] / df_county_total['n2_outflow']
df_county_total.to_csv('county_total_ALL.csv')

# individual county migration - this will be used for single-county select
df_county_to_county2 = df_total[~df_total['unique_tag'].str.contains('total migration-')]
df_county_to_county2['arc_county'] = df_county_to_county2['unique_tag'].str.split('-').str[0]
df_county_to_county2['unique_tag2'] = df_county_to_county2['unique_tag'].str[:-5]
df_county_to_county2.to_csv('total_tester.csv')
df_county_to_county = df_county_to_county2.copy()
df_county_to_county[['prim_county', 'sec_county', 'sec_county_state', 'yr_range2', 'idk']] = \
df_county_to_county['unique_tag'].str.split('-', expand=True)

atl_counties = list(fips_dict.keys())
df_county_to_county = df_county_to_county[~df_county_to_county['sec_county'].isin(atl_counties)]

df_county_to_county['sec_unique'] = df_county_to_county['sec_county'] + ', ' + df_county_to_county['sec_county_state']

df_county_grouped = df_county_to_county.groupby(['yr_range', 'sec_unique']).agg({
    'year1':'mean',
    'year2':'mean',
    'n2_inflow':'sum',
    'agi_inflow':'sum',
    'n2_outflow':'sum',
    'agi_outflow':'sum',
    'n2_net':'sum',
    'agi_net':'sum'
}).reset_index()

df_county_grouped.to_csv('county_to_county_ALL.csv')

print('export complete!')

export complete!


/Users/kcdeleon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/kcdeleon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/kcdeleon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [8]:
gdf = gpd.read_file('oregon.geojson.json')
gdf = gdf[['NAME10', 'NAMELSAD10', 'GEOID10', 'geometry']]

# county totals
df = pd.read_csv('county_total_ALL.csv')
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
df['ore_county'] = df['unique_tag'].map(lambda x: x.replace(' total migration-', ""))
df['ore_county'] = df['ore_county'].str[:-4]

df_joined = gdf.merge(df, right_on='ore_county', left_on='NAMELSAD10')
df_joined = df_joined.drop(columns=['NAMELSAD10','NAME10','n2_net','agi_net'])
df_joined.to_file('county_total_MERGED.gpkg')

/Users/kcdeleon/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



In [12]:
m = folium.Map(location=[45.5198101,-122.7069389], zoom_start=9, tiles=None)

mapbox_token = 'pk.eyJ1Ijoia2NkZWxlb24iLCJhIjoiY2xpbzE3aW9qMTZzeTNqbnp0NzVzZjAwMyJ9.6tJZUV9QSK-lMvgMP67Zng'

# base layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=' + mapbox_token,
    attr = 'Mapbox',
    name = 'Streets',
    overlay = False,
    control = False,
    #show = True,
    min_zoom = 8,
    max_zoom = 30
    ).add_to(m)

# variables
full_years = [2017, 2018, 2019, 2020, 2021]
mig_direction = 'In'
direction_lower = 'in'

# "go read the data..."
df_counties = pd.read_csv('county_total_ALL.csv')
df_counties['ore_county'] = df_counties['unique_tag'].map(lambda x: x.replace(' total migration-', ""))
df_counties['ore_county'] = df_counties['ore_county'].str[:-4]
df_counties = df_counties[(df_counties['year1'].isin(full_years)) & (df_counties['year2'].isin(full_years))]
df_counties = df_counties.groupby('ore_county').sum().reset_index()
df_counties['agi_capita_in'] = df_counties['agi_inflow'] / df_counties['n2_inflow']
df_counties['agi_capita_out'] = df_counties['agi_outflow'] / df_counties['n2_outflow']

gdf = gpd.read_file('oregon.geojson.json')

gdf2 = gdf.merge(df_counties, left_on='NAMELSAD10', right_on='ore_county')
gdf2['agi_capita_in'] = gdf2['agi_capita_in'].astype(int)
gdf2['agi_capita_out'] = gdf2['agi_capita_out'].astype(int)

gdf2['county_tooltip'] = gdf2['ore_county'].str.split(' ').str[0]
gdf2['in_tooltip'] = gdf2['agi_capita_in'].apply(lambda x: "${:,.0f}".format(x))
gdf2['out_tooltip'] = gdf2['agi_capita_out'].apply(lambda x: "${:,.0f}".format(x))

choro_dict_value = {
    'In':'agi_capita_in',
    'Out':'agi_capita_out'
}

choro_dict_tooltip = {
    'In':'in_tooltip',
    'Out':'out_tooltip'
}

# choropleth
delta_choro = folium.Choropleth(
    geo_data=gdf2,
    data=gdf2,
    columns=['ore_county', choro_dict_value[mig_direction], 'county_tooltip', 'in_tooltip'],
    bins=4,  
    key_on='feature.properties.ore_county', 
    fill_color='Greens',
    name='AGI per capita',
    fill_opacity=0.6,
    nan_fill_color="Black", 
    line_opacity=0.2,
    highlight=True,
    show=True,
    control=False,
    line_color='black').add_to(m)

# tooltip
delta_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [
        'county_tooltip', 
        choro_dict_tooltip[mig_direction]
        ], 
    aliases = [
        'County:', 
        f"Average AGI per capita {direction_lower}flow: "
        ],
    style = ('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True
    ))

# remove stock legend from map
for key in delta_choro._children:
    if key.startswith('color_map'):
        del(delta_choro._children[key])
delta_choro.add_to(m)

m

AttributeError: module 'folium' has no attribute 'Choropleth'